In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from source import Vermont_Farm
from statsmodels.compat import lzip
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
from stargazer.stargazer import Stargazer

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
VF = Vermont_Farm()
# df = pd.read_csv('./1850final_w_TotalOutput.csv')

# Revenue per Land as dependent variable

In [4]:
df_diversity = pd.read_csv('1850final.csv')
df_diversity = df_diversity.drop(['Name', 'town', 'unimproved', 'cash_val', 'produce_val', 'pg_num', 'year', 'rown'], axis=1)
df_diversity.head(1)

,improved,machine_val,horse,mules,milkcow,oxen,cattle,sheep,swine,livestock_val,...,hemp_waterrot,flax,flaxseed,silk,maplesugar,canesugar,molasses,beewax,manufactures_val,animals_val
0,48,100,3,0,4,0,0,0,2,450,...,0,0,0,0,0,0,0,0,0,40


In [5]:
df_diversity = df_diversity.to_numpy()
print(df_diversity.shape)
df_diversity = 1 * (df_diversity > 0)
print(df_diversity.shape)

(170, 42)
(170, 42)


In [6]:
df3 = pd.read_csv('./1850final_w_TotalOutput.csv')
div_score = np.sum(df_diversity, axis=1)
print(div_score.shape)
df3['diversity_score'] = div_score

(170,)


In [7]:
entropy_score = df3["entropy_score"].to_numpy()
df3['diversity_squared'] = np.square(div_score)
df3['e_to_diversity'] = np.exp(div_score)
df3['entropy_squared'] = np.square(entropy_score)

# generated diversity score

In [8]:
cols2 = ['improved', 'unimproved', 
       'machine_val', 'horse', 'mules']
# Fit regression model (using the natural log of one of the regressors)
# results = smf.ols("Lottery ~ Literacy + np.log(Pop1831)", data=dat).fit()
reg1 = smf.ols("revenue_per_land ~ diversity_score", data=df3).fit()
# print(reg1.summary())
reg2 = smf.ols("revenue_per_land ~ diversity_score + improved + unimproved", data=df3).fit()
print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:       revenue_per_land   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     2.605
Date:                Wed, 17 May 2023   Prob (F-statistic):             0.0536
Time:                        22:08:30   Log-Likelihood:                -917.95
No. Observations:                 170   AIC:                             1844.
Df Residuals:                     166   BIC:                             1856.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          39.7288     31.098     

In [19]:
reg3 = smf.ols("revenue_per_land ~ diversity_score + diversity_squared + improved + unimproved", data=df3).fit()
reg4 = smf.ols("revenue_per_land ~ entropy_score + improved + unimproved", data=df3).fit()
reg5 = smf.ols("revenue_per_land ~ entropy_score + entropy_squared + improved + unimproved", data=df3).fit()
reg6 = smf.ols("revenue_per_land ~ entropy_score + machine_val + cash_val", data=df3).fit()
reg7 = smf.ols("revenue_per_land ~ entropy_score + machine_val + cash_val + livestock_val + animals_val", data=df3).fit()
print(reg7.summary())

                            OLS Regression Results                            
Dep. Variable:       revenue_per_land   R-squared:                       0.223
Model:                            OLS   Adj. R-squared:                  0.199
Method:                 Least Squares   F-statistic:                     9.393
Date:                Wed, 17 May 2023   Prob (F-statistic):           6.88e-08
Time:                        22:11:26   Log-Likelihood:                -900.46
No. Observations:                 170   AIC:                             1813.
Df Residuals:                     164   BIC:                             1832.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        81.4321     15.850      5.138

In [20]:
print(reg6.summary())

                            OLS Regression Results                            
Dep. Variable:       revenue_per_land   R-squared:                       0.202
Model:                            OLS   Adj. R-squared:                  0.188
Method:                 Least Squares   F-statistic:                     14.04
Date:                Wed, 17 May 2023   Prob (F-statistic):           3.36e-08
Time:                        22:11:27   Log-Likelihood:                -902.65
No. Observations:                 170   AIC:                             1813.
Df Residuals:                     166   BIC:                             1826.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        73.1976     15.444      4.740

In [21]:
stargazer = Stargazer([reg1, reg2, reg3, reg4, reg5, reg7])

In [22]:
print(stargazer.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Intercept & 49.626$^{}$ & 39.729$^{}$ & -203.316$^{}$ & 124.395$^{***}$ & 106.805$^{***}$ & 81.432$^{***}$ \\
  & (30.967) & (31.098) & (181.676) & (14.193) & (34.288) & (15.850) \\
 animals_val & & & & & & -0.037$^{}$ \\
  & & & & & & (0.047) \\
 cash_val & & & & & & -0.018$^{***}$ \\
  & & & & & & (0.003) \\
 diversity_score & 1.986$^{}$ & 3.219$^{*}$ & 31.172$^{}$ & & & \\
  & (1.674) & (1.750) & (20.661) & & & \\
 diversity_squared & & & -0.788$^{}$ & & & \\
  & & & (0.581) & & & \\
 entropy_score & & & & -33.190$^{**}$ & 8.591$^{}$ & -8.914$^{}$ \\
  & & & & (14.611) & (75.537) & (16.478) \\
 entropy_squared & & & & & -22.975$^{}$ & \\
  & & & & & (40.750) & \\
 improved & & -0.164$^{**}$ & -0.160$^{**}$ & -0.118$^{*}$ & -0.114$^{*}$ &